<a href="https://colab.research.google.com/github/imthelizardking/CMP784--Final--Project/blob/main/suggestion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and install libraries, dependencies

In [2]:
import tensorflow as tf
import numpy as np
import os
import json

Connect to gdrive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import data

In [4]:
# import data, from gdrive
dirJson = "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs/"
%cd "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs"

/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs


TO-DO: Put labels onto one hot encode vector rows

In [19]:
json_files = [pos_json for pos_json in os.listdir(dirJson) if pos_json.endswith('.json')]
NUM_FEATURES = 3
labels_vector = np.zeros((len(json_files),NUM_FEATURES))
labels = []
counter_outer = 0
for json_ in json_files:
  jsonFile = open(json_, 'r')
  values = json.load(jsonFile)
  jsonFile.close()
  labels.clear()
  counter_inner = 0
  for (k,v) in values.items():
    if k.startswith('item'):
      labels.append(v["category_name"])
      if v["category_name"]=="vest":
        labels_vector[counter_outer][0] = 1
      elif v["category_name"]=="sling dress":
        labels_vector[counter_outer][1] = 1
      elif v["category_name"]=="short sleeve dress":
        labels_vector[counter_outer][2] = 1        
  print(labels)  
  counter_outer = counter_outer + 1
print(labels_vector)

['sling dress', 'vest']
['short sleeve dress']
['sling dress', 'vest']
[[1. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]]


Trainer:

In [ ]:
def Trainer(x, weights, biases):
  

Set Trainer Parameters and Train

In [ ]:
# trainer param.s an go

Test predictor

In [ ]:
# test predictor